# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [50]:
# your code here
import numpy as np
import pandas as pd

tweets = pd.read_csv('../sentiment140.csv', encoding='latin-1', header = None)
tweets.columns = ['target', 'ids', 'date', 'flag', 'user', 'text']
tweets_sub = tweets.sample(5000)

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [51]:
# your code here
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import re

def clean_up(s):
    new = re.sub(r'http\S+', ' ', s).replace("'", " ").lower()
    new = re.sub(r"[^a-zA-Z ]", " ", new)
    return new

def tokenize(s):
    new = nltk.word_tokenize(s)
    return new

def stem_and_lemmatize(l):
    ps = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    new = [lemmatizer.lemmatize(ps.stem(w)) for w in l]
    return new

def remove_stopwords(l):
    stop_words = stopwords.words('english')
    new = [x for x in l if x not in stop_words]
    return new

In [52]:
tweets_sub['text_processed'] = tweets_sub.apply(lambda x : remove_stopwords(stem_and_lemmatize(tokenize(clean_up(x['text'])))), axis = 1)
tweets_sub.head()

,target,ids,date,flag,user,text,text_processed
856254,4,1573717854,Tue Apr 21 02:31:14 PDT 2009,NO_QUERY,alphyne,@tintinnya: i can see clearly now..the rain is...,"[tintinnya, see, clearli, rain, gone]"
413849,0,2060684714,Sat Jun 06 19:22:41 PDT 2009,NO_QUERY,timhoeck,Not having much programming luck this weekend,"[much, program, luck, thi, weekend]"
873659,4,1679731504,Sat May 02 10:02:57 PDT 2009,NO_QUERY,clicheunreal,just woke up....had the best dream ever! ? ht...,"[woke, best, dream, ever]"
465989,0,2175466917,Mon Jun 15 01:04:02 PDT 2009,NO_QUERY,AggyB,@pareidoliac no But i made sure his profile g...,"[pareidoliac, made, sure, hi, profil, got, delet]"
1183414,4,1982417854,Sun May 31 11:10:05 PDT 2009,NO_QUERY,happymonkeyshoe,@johnhood I'm after your Big Fish....,"[johnhood, big, fish]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [53]:
# your code here

from nltk import FreqDist
all_words = [x for lst in tweets_sub['text_processed'] for x in lst]
all_words
x = nltk.FreqDist(all_words)
mosts = [y[0] for y in x.most_common(500)]
mosts

['go',
 'get',
 'day',
 'wa',
 'thi',
 'good',
 'work',
 'like',
 'quot',
 'love',
 'time',
 'got',
 'today',
 'want',
 'u',
 'lol',
 'thank',
 'one',
 'miss',
 'back',
 'think',
 'see',
 'realli',
 'know',
 'im',
 'feel',
 'night',
 'hope',
 'amp',
 'come',
 'ha',
 'watch',
 'well',
 'new',
 'make',
 'na',
 'oh',
 'need',
 'still',
 'wait',
 'home',
 'morn',
 'last',
 'great',
 'sleep',
 'fun',
 'much',
 'wish',
 'look',
 'right',
 'tri',
 'whi',
 'twitter',
 'tomorrow',
 'way',
 'sad',
 'haha',
 'tweet',
 'would',
 'friend',
 'week',
 'bad',
 'follow',
 'thing',
 'onli',
 'take',
 'nice',
 'tonight',
 'sorri',
 'could',
 'happi',
 'gon',
 'yeah',
 'say',
 'though',
 'even',
 'ye',
 'hi',
 'year',
 'bed',
 'hate',
 'let',
 'awesom',
 'hour',
 'littl',
 'rain',
 'first',
 'dont',
 'next',
 'use',
 'girl',
 'veri',
 'movi',
 'final',
 'soon',
 'never',
 'sure',
 'better',
 'life',
 'ok',
 'peopl',
 'guy',
 'wan',
 'hey',
 'start',
 'call',
 'hurt',
 'live',
 'help',
 'best',
 'old',
 'a

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [54]:
# your code here
def find_features(document):
    words = set(document)
    features = {}
    for w in mosts:
        features[w] = (w in words)
    return features

#find_features(tweets_sub['text_processed'][0])

In [55]:
documents = list(zip(tweets_sub['text_processed'], np.where(tweets_sub['target'] == 4, True, False)))

featuresets = [(find_features(rev), category) for (rev, category) in documents]
featuresets[0]

({'go': False,
  'get': False,
  'day': False,
  'wa': False,
  'thi': False,
  'good': False,
  'work': False,
  'like': False,
  'quot': False,
  'love': False,
  'time': False,
  'got': False,
  'today': False,
  'want': False,
  'u': False,
  'lol': False,
  'thank': False,
  'one': False,
  'miss': False,
  'back': False,
  'think': False,
  'see': True,
  'realli': False,
  'know': False,
  'im': False,
  'feel': False,
  'night': False,
  'hope': False,
  'amp': False,
  'come': False,
  'ha': False,
  'watch': False,
  'well': False,
  'new': False,
  'make': False,
  'na': False,
  'oh': False,
  'need': False,
  'still': False,
  'wait': False,
  'home': False,
  'morn': False,
  'last': False,
  'great': False,
  'sleep': False,
  'fun': False,
  'much': False,
  'wish': False,
  'look': False,
  'right': False,
  'tri': False,
  'whi': False,
  'twitter': False,
  'tomorrow': False,
  'way': False,
  'sad': False,
  'haha': False,
  'tweet': False,
  'would': False,
  'frie

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [56]:
# your code here
n = int(len(featuresets) / 2)
training_set = featuresets[:n]
testing_set = featuresets[n:]
classifier = nltk.NaiveBayesClassifier.train(training_set)
classifier.show_most_informative_features(20)

Most Informative Features
                     die = True            False : True   =     13.9 : 1.0
                     ani = True            False : True   =     12.0 : 1.0
                     sad = True            False : True   =     11.5 : 1.0
                    bore = True            False : True   =      9.1 : 1.0
                     doe = True            False : True   =      9.1 : 1.0
                      pm = True            False : True   =      8.5 : 1.0
                  anymor = True            False : True   =      8.5 : 1.0
                    poor = True            False : True   =      7.1 : 1.0
                    hurt = True            False : True   =      6.8 : 1.0
                    cant = True            False : True   =      6.7 : 1.0
                    damn = True            False : True   =      6.7 : 1.0
                     wow = True             True : False  =      6.5 : 1.0
                   thank = True             True : False  =      6.4 : 1.0

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [57]:
# your code here
nltk.classify.accuracy(classifier, testing_set)

0.6992

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here


## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

# your code here
- Refer to nlp-function file

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

# your code here
